In [ ]:
import pandas as pd
import random
import numpy as np
import string
import re
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from sklearn.ensemble import RandomForestClassifier
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
tweets = pd.read_csv('/content/drive/MyDrive/sample_dataset/tweet_aug.csv')
tweets

,Unnamed: 0,sentimen,tweet
0,0,negatif,kata prabowo indonesia tidak harga bangsa asin...
1,1,negatif,kata prabowo indonesia tidak harga bangsa asin...
2,2,netral,batu langka tasbih jokowi hadiah habib luthfi ...
3,3,netral,batu langka rosario jokowi hadiah habib luthfi...
4,4,netral,era jokowi ekonomi indonesia makin baik maju m...
...,...,...,...
3625,3625,netral,bangun bangsa dukung ekonomi negara sama bapak...
3626,3626,netral,bantu maju ekonomi bangsa sama bapak jokowi yuk
3627,3627,netral,bantu kedepan ekonomi bangsa sama bapak jokowi...
3628,3628,netral,bapak jokowi ubah cara pandang ekonomi kini se...


In [ ]:
del tweets[tweets.columns[0]]

In [ ]:
tweets

,sentimen,tweet
0,negatif,kata prabowo indonesia tidak harga bangsa asin...
1,negatif,kata prabowo indonesia tidak harga bangsa asin...
2,netral,batu langka tasbih jokowi hadiah habib luthfi ...
3,netral,batu langka rosario jokowi hadiah habib luthfi...
4,netral,era jokowi ekonomi indonesia makin baik maju m...
...,...,...
3625,netral,bangun bangsa dukung ekonomi negara sama bapak...
3626,netral,bantu maju ekonomi bangsa sama bapak jokowi yuk
3627,netral,bantu kedepan ekonomi bangsa sama bapak jokowi...
3628,netral,bapak jokowi ubah cara pandang ekonomi kini se...


In [ ]:
tweets['sentimen'].value_counts()

positif    1224
netral     1214
negatif    1192
Name: sentimen, dtype: int64

In [ ]:
mapping = {
    "negatif": 0,
    "netral": 1,
    "positif": 2,
}

tweets['sentimen'] = [mapping[label] for label in tweets['sentimen']]
tweets

,sentimen,tweet
0,0,kata prabowo indonesia tidak harga bangsa asin...
1,0,kata prabowo indonesia tidak harga bangsa asin...
2,1,batu langka tasbih jokowi hadiah habib luthfi ...
3,1,batu langka rosario jokowi hadiah habib luthfi...
4,1,era jokowi ekonomi indonesia makin baik maju m...
...,...,...
3625,1,bangun bangsa dukung ekonomi negara sama bapak...
3626,1,bantu maju ekonomi bangsa sama bapak jokowi yuk
3627,1,bantu kedepan ekonomi bangsa sama bapak jokowi...
3628,1,bapak jokowi ubah cara pandang ekonomi kini se...


# Split Data

In [ ]:
tweets['tweet'] = tweets['tweet'].astype(str)
tweets['tweet'] = tweets['tweet'].fillna('')

In [ ]:
features = tweets['tweet']
target = tweets['sentimen']

In [ ]:
X = features
Y = target
# display(X)
# display(Y)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
train_tweet, test_tweet, train_sentimen, test_sentimen = train_test_split(tweets['tweet'].to_numpy(), tweets['sentimen'].to_numpy(), test_size=0.2, random_state=0)

In [ ]:
round(sum([len(i.split()) for i in train_tweet])/len(train_tweet))

19

In [ ]:
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 19 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    standardize="lower_and_strip_punctuation",
                                     split="whitespace",
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
text_vectorizer

In [ ]:
text_vectorizer.adapt(X_train.to_numpy())

In [ ]:
sample_setence = X_train[0]
text_vectorizer([sample_setence])

<tf.Tensor: shape=(1, 19), dtype=int64, numpy=
array([[  41,    4,   10,    5,    8,   66,  652,  407,  336,   33,  200,
          26,   54, 1965,    0,    0,    0,    0,    0]])>

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:10] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-10:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 5658
Top 5 most common words: ['', '[UNK]', 'ekonomi', 'jokowi', 'prabowo', 'tidak', 'bapak', 'gaji', 'harga', 'salah']
Bottom 5 least common words: ['absolut', 'abshar', 'absah', 'abg', 'abdul', 'abdalla', 'abah', 'abadi', 'aaamiin', 'aa']


# Create Embedding

In [ ]:
tf.random.set_seed(42)

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=64, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

embedding

In [ ]:
# Get a random sentence from training set
random_sentence = random.choice(X_train)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")
# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
moga periode dua masa perintah bapak jokowi maruf amin nu bijak subsidi harga obatx an kualitas paten banyak rakyat sakit lekas sembuh kerja lagigo      

Embedded version:


<tf.Tensor: shape=(1, 19, 64), dtype=float32, numpy=
array([[[ 0.02666571, -0.03184249,  0.01886283, ..., -0.03305317,
          0.02769513, -0.01963159],
        [-0.04473915,  0.01595577, -0.01489934, ...,  0.02700246,
         -0.01645341, -0.02294894],
        [ 0.00646809, -0.04199778,  0.03767428, ...,  0.04471574,
         -0.00533433, -0.03236911],
        ...,
        [-0.04620086, -0.03563418,  0.04979349, ...,  0.04895489,
          0.01445272,  0.00196796],
        [-0.03436221, -0.03728436,  0.02942674, ...,  0.04428313,
         -0.04122413, -0.01722555],
        [ 0.02654687, -0.01429148, -0.02657776, ...,  0.04800484,
          0.01290656, -0.0479096 ]]], dtype=float32)>

In [ ]:
def calculate_results(y_true, y_pred):

  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

# Model 1

In [ ]:
def model1():

  inputs = layers.Input(shape=(1,), dtype="string")
  x = text_vectorizer(inputs)
  layer = embedding(x)
  layer = layers.LSTM(32)(layer)
  layer = layers.Dense(3, name='out_layer')(layer)
  layer = layers.Activation('softmax')(layer)
  model = tf.keras.Model(inputs=inputs, outputs=layer, name="model_1_LSTM")
  return model

In [ ]:
lstm1 = model1()
lstm1.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
lstm1.summary()

Model: "model_1_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (Text  (None, 19)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 19, 64)            640000    
                                                                 
 lstm_2 (LSTM)               (None, 32)                12416     
                                                                 
 out_layer (Dense)           (None, 3)                 99        
                                                                 
 activation_2 (Activation)   (None, 3)                 0         
                                                      

In [ ]:
lstm1_history = lstm1.fit(X_train, np.float32(Y_train),
                          epochs=10,
                          batch_size=128,
                          validation_data=(X_test, np.float32(Y_test)),)

Epoch 1/10
23/23 [==============================] - 4s 62ms/step - loss: 1.0770 - accuracy: 0.4504 - val_loss: 1.0523 - val_accuracy: 0.4421
Epoch 2/10
23/23 [==============================] - 1s 45ms/step - loss: 0.9773 - accuracy: 0.5623 - val_loss: 0.9215 - val_accuracy: 0.6047
Epoch 3/10
23/23 [==============================] - 1s 46ms/step - loss: 0.6511 - accuracy: 0.7528 - val_loss: 0.7096 - val_accuracy: 0.7107
Epoch 4/10
23/23 [==============================] - 1s 48ms/step - loss: 0.3501 - accuracy: 0.8846 - val_loss: 0.6272 - val_accuracy: 0.7645
Epoch 5/10
23/23 [==============================] - 1s 46ms/step - loss: 0.1797 - accuracy: 0.9494 - val_loss: 0.5654 - val_accuracy: 0.8140
Epoch 6/10
23/23 [==============================] - 1s 28ms/step - loss: 0.1026 - accuracy: 0.9714 - val_loss: 0.6060 - val_accuracy: 0.8223
Epoch 7/10
23/23 [==============================] - 1s 30ms/step - loss: 0.0677 - accuracy: 0.9831 - val_loss: 0.5941 - val_accuracy: 0.8306
Epoch 8/10
23

In [ ]:
test_sentimen

array([1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2, 1, 2,
       0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 1, 1, 0, 0, 2, 0, 2, 0, 2, 1, 1, 2,
       0, 1, 1, 0, 1, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 0,
       1, 0, 1, 1, 2, 2, 0, 2, 1, 1, 0, 1, 2, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       2, 0, 2, 0, 0, 2, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 2, 0, 1,
       0, 2, 0, 1, 0, 2, 1, 0, 1, 2, 2, 0, 1, 2, 1, 1, 1, 0, 2, 1, 0, 2,
       0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 0, 1, 2,
       0, 0, 0, 1, 0, 1, 2, 2, 1, 1, 2, 1, 1, 0, 2, 1, 2, 1, 2, 2, 0, 1,
       0, 1, 2, 0, 2, 1, 2, 2, 2, 1, 2, 0, 2, 1, 1, 1, 1, 2, 0, 2, 1, 0,
       1, 0, 2, 2, 0, 2, 1, 0, 2, 2, 1, 1, 1, 0, 1, 1, 1, 2, 2, 2, 1, 2,
       2, 2, 0, 2, 1, 0, 2, 0, 2, 1, 1, 0, 2, 1, 2, 1, 2, 1, 0, 0, 2, 0,
       2, 1, 1, 0, 1, 0, 1, 1, 0, 0, 2, 0, 2, 1, 0, 0, 0, 2, 1, 2, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 2, 2, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 1, 0, 1,
       0, 2, 2, 2, 1, 0, 0, 2, 0, 2, 1, 0, 0, 2, 0,

In [ ]:
# Make predictions on the validation dataset
model_1_pred_probs = lstm1.predict(test_tweet)
display(model_1_pred_probs.shape, model_1_pred_probs[:10]) # view the first 10

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds = np.argmax(model_1_preds, axis=1)

# Calculate LSTM model results
model_1_results = calculate_results(y_true=test_sentimen,
                                    y_pred=model_1_preds)
display(model_1_results)

display(tf.squeeze(tf.round(lstm1.predict(np.array(['Pak @sandiuno dan pak @prabowo Gajinya kirim ke saya sj'])))).numpy())

23/23 [==============================] - 1s 5ms/step


(726, 3)

array([[0.04708164, 0.8743616 , 0.07855678],
       [0.00243505, 0.9936584 , 0.0039065 ],
       [0.78725564, 0.1598934 , 0.05285095],
       [0.00179452, 0.9958457 , 0.00235982],
       [0.00115742, 0.99703795, 0.0018046 ],
       [0.00206428, 0.00199937, 0.9959364 ],
       [0.00128542, 0.99640846, 0.00230596],
       [0.00100555, 0.9975449 , 0.00144963],
       [0.00174058, 0.00293103, 0.9953285 ],
       [0.00591882, 0.9867927 , 0.00728849]], dtype=float32)

{'accuracy': 83.33333333333334,
 'precision': 0.8353259523450698,
 'recall': 0.8333333333333334,
 'f1': 0.8335245790079424}

1/1 [==============================] - 0s 25ms/step


array([0., 1., 0.], dtype=float32)

In [ ]:
predicted_sentiment = []

for idx in range(len(test_tweet)):
  predicted = tf.squeeze(tf.round(lstm1.predict(np.array([test_tweet[idx]])))).numpy()
  predicted_sentiment.append(predicted)

1/1 [==============================] - 0s 24ms/step


In [ ]:
predicted_sentiment

[array([0., 1., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 

In [ ]:
predicted_dataframe = pd.DataFrame(columns=['sentimen', 'predicted_sentimen', 'tweet'])

In [ ]:
predicted_dataframe['sentimen'] = test_sentimen
predicted_dataframe['predicted_sentimen'] = [np.argmax(array) for array in predicted_sentiment]
predicted_dataframe['tweet'] = test_tweet

display(predicted_dataframe)

,sentimen,predicted_sentimen,tweet
0,1,1,mau tanya impact buat negara kek gimana kalo g...
1,1,1,jokowi raja salman sepakat tingkat kerja sama ...
2,0,0,kala tax amnesty kemarin prabowo sama sandiaga...
3,1,1,gimana nih apa gaji tidak jokowi prabowo
4,1,1,beranganangan gaji dokter binatang minimal jut...
...,...,...,...
721,0,0,nyerocos gurubesar sumitro joyohadikusumo ekon...
722,1,1,asosiasi usaha bumiputera nusantara se indones...
723,0,0,tidak masuk otak lu bapak jokowi nanya ekonomi...
724,0,0,eh abang beda startup monetizing hobi monetize...


# Model 2

In [ ]:
def model2():

  inputs = layers.Input(shape=(1,), dtype="string")
  x = text_vectorizer(inputs)
  layer = embedding(x)
  layer = layers.LSTM(64)(layer)
  layer = layers.Dense(256, name='FC1')(layer)
  layer = layers.Activation('relu')(layer)
  layer = layers.Dropout(0.5)(layer)
  layer = layers.Dense(3, name='out_layer')(layer)
  layer = layers.Activation('softmax')(layer)
  model = tf.keras.Model(inputs=inputs, outputs=layer)
  return model

In [ ]:
lstm2 = model2()
lstm2.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
lstm2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (Text  (None, 19)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 19, 64)            640000    
                                                                 
 lstm_4 (LSTM)               (None, 64)                33024     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_5 (Activation)   (None, 256)               0         
                                                           

In [ ]:
lstm2_history = lstm2.fit(X_train, np.float32(Y_train),
                          epochs=10,
                          batch_size=256,
                          validation_data=(X_test, np.float32(Y_test)),)

Epoch 1/10
12/12 [==============================] - 1s 122ms/step - loss: 0.0113 - accuracy: 0.9966 - val_loss: 0.7715 - val_accuracy: 0.8581
Epoch 2/10
12/12 [==============================] - 1s 119ms/step - loss: 0.0098 - accuracy: 0.9972 - val_loss: 0.7923 - val_accuracy: 0.8567
Epoch 3/10
12/12 [==============================] - 1s 123ms/step - loss: 0.0087 - accuracy: 0.9972 - val_loss: 0.8787 - val_accuracy: 0.8540
Epoch 4/10
12/12 [==============================] - 1s 74ms/step - loss: 0.0083 - accuracy: 0.9976 - val_loss: 0.8800 - val_accuracy: 0.8526
Epoch 5/10
12/12 [==============================] - 1s 68ms/step - loss: 0.0077 - accuracy: 0.9976 - val_loss: 0.9048 - val_accuracy: 0.8554
Epoch 6/10
12/12 [==============================] - 1s 74ms/step - loss: 0.0085 - accuracy: 0.9972 - val_loss: 0.8994 - val_accuracy: 0.8595
Epoch 7/10
12/12 [==============================] - 1s 69ms/step - loss: 0.0082 - accuracy: 0.9976 - val_loss: 0.9026 - val_accuracy: 0.8595
Epoch 8/10

In [ ]:
# Make predictions on the validation dataset
model_2_pred_probs = lstm2.predict(test_tweet)
display(model_2_pred_probs.shape, model_2_pred_probs[:10]) # view the first 10

model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds = np.argmax(model_2_preds, axis=1)

# Calculate LSTM model results
model_2_results = calculate_results(y_true=test_sentimen,
                                    y_pred=model_2_preds)
display(model_2_results)

display(tf.squeeze(tf.round(lstm2.predict(np.array(['Pak @sandiuno dan pak @prabowo Gajinya kirim ke saya sj'])))).numpy())

23/23 [==============================] - 0s 7ms/step


(726, 3)

array([[1.6701415e-02, 9.4365978e-01, 3.9638881e-02],
       [1.1204571e-03, 9.8630613e-01, 1.2573286e-02],
       [9.9960488e-01, 2.6227301e-04, 1.3287395e-04],
       [1.0831840e-04, 9.9981165e-01, 7.9987745e-05],
       [8.7483249e-06, 9.9997431e-01, 1.6878268e-05],
       [1.4479563e-05, 6.9098758e-05, 9.9991637e-01],
       [1.9385901e-05, 9.9988180e-01, 9.8707984e-05],
       [5.3007734e-06, 9.9998432e-01, 1.0356511e-05],
       [1.6269238e-05, 2.6498133e-04, 9.9971879e-01],
       [1.9358615e-04, 9.9963629e-01, 1.7008065e-04]], dtype=float32)

{'accuracy': 85.81267217630854,
 'precision': 0.8624572755484073,
 'recall': 0.8581267217630854,
 'f1': 0.8587079309277267}

1/1 [==============================] - 0s 29ms/step


array([1., 0., 0.], dtype=float32)

In [ ]:
predicted_sentiment = []

for idx in range(len(test_tweet)):
  predicted = tf.squeeze(tf.round(lstm2.predict(np.array([test_tweet[idx]])))).numpy()
  predicted_sentiment.append(predicted)

1/1 [==============================] - 0s 36ms/step


In [ ]:
predicted_dataframe['sentimen'] = test_sentimen
predicted_dataframe['predicted_sentimen'] = [np.argmax(array) for array in predicted_sentiment]
predicted_dataframe['tweet'] = test_tweet

display(predicted_dataframe)

,sentimen,predicted_sentimen,tweet
0,1,1,mau tanya impact buat negara kek gimana kalo g...
1,1,1,jokowi raja salman sepakat tingkat kerja sama ...
2,0,0,kala tax amnesty kemarin prabowo sama sandiaga...
3,1,1,gimana nih apa gaji tidak jokowi prabowo
4,1,1,beranganangan gaji dokter binatang minimal jut...
...,...,...,...
721,0,0,nyerocos gurubesar sumitro joyohadikusumo ekon...
722,1,1,asosiasi usaha bumiputera nusantara se indones...
723,0,0,tidak masuk otak lu bapak jokowi nanya ekonomi...
724,0,0,eh abang beda startup monetizing hobi monetize...


# Model 3

In [ ]:
def model3():

  inputs = layers.Input(shape=(1,), dtype="string")
  x = text_vectorizer(inputs)
  layer = embedding(x)
  layer = layers.LSTM(64)(layer)
  layer = layers.Dense(256, name='FC1')(layer)
  layer = layers.Dense(128,)(layer)
  layer = layers.Activation('relu')(layer)
  layer = layers.Dropout(0.5)(layer)
  layer = layers.Dense(3, name='out_layer')(layer)
  layer = layers.Activation('softmax')(layer)
  model = tf.keras.Model(inputs=inputs, outputs=layer)
  return model

In [ ]:
lstm3 = model3()
lstm3.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
lstm3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (Text  (None, 19)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 19, 64)            640000    
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                           

In [ ]:
lstm3_history = lstm3.fit(X_train, np.float32(Y_train),
                          epochs=10,
                          batch_size=128,
                          validation_data=(X_test, np.float32(Y_test)),)

Epoch 1/10
23/23 [==============================] - 6s 123ms/step - loss: 0.7767 - accuracy: 0.7548 - val_loss: 0.4866 - val_accuracy: 0.8237
Epoch 2/10
23/23 [==============================] - 1s 54ms/step - loss: 0.0578 - accuracy: 0.9852 - val_loss: 1.2488 - val_accuracy: 0.8485
Epoch 3/10
23/23 [==============================] - 1s 53ms/step - loss: 0.0424 - accuracy: 0.9862 - val_loss: 1.0808 - val_accuracy: 0.8264
Epoch 4/10
23/23 [==============================] - 1s 53ms/step - loss: 0.0191 - accuracy: 0.9931 - val_loss: 0.8872 - val_accuracy: 0.8375
Epoch 5/10
23/23 [==============================] - 1s 52ms/step - loss: 0.0170 - accuracy: 0.9941 - val_loss: 0.8858 - val_accuracy: 0.8581
Epoch 6/10
23/23 [==============================] - 1s 51ms/step - loss: 0.0112 - accuracy: 0.9969 - val_loss: 0.9449 - val_accuracy: 0.8402
Epoch 7/10
23/23 [==============================] - 1s 52ms/step - loss: 0.0126 - accuracy: 0.9959 - val_loss: 0.8593 - val_accuracy: 0.8554
Epoch 8/10
2

In [ ]:
# Make predictions on the validation dataset
model_3_pred_probs = lstm3.predict(test_tweet)
display(model_3_pred_probs.shape, model_3_pred_probs[:10]) # view the first 10

model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds = np.argmax(model_3_preds, axis=1)

# Calculate LSTM model results
model_3_results = calculate_results(y_true=test_sentimen,
                                    y_pred=model_3_preds)
display(model_3_results)

display(tf.squeeze(tf.round(lstm3.predict(np.array(['Pak @sandiuno dan pak @prabowo Gajinya kirim ke saya sj'])))).numpy())

23/23 [==============================] - 1s 8ms/step


(726, 3)

array([[2.3131063e-02, 9.2861521e-01, 4.8253674e-02],
       [3.6192462e-03, 9.7578192e-01, 2.0598857e-02],
       [9.9996608e-01, 3.2703319e-05, 1.1943067e-06],
       [4.5091041e-05, 9.9990791e-01, 4.6905829e-05],
       [7.7251497e-07, 9.9999589e-01, 3.3959070e-06],
       [5.0832182e-07, 5.8487120e-05, 9.9994093e-01],
       [8.9333270e-07, 9.9996358e-01, 3.5509358e-05],
       [2.3942411e-07, 9.9999827e-01, 1.3968238e-06],
       [3.3725637e-08, 9.4950383e-06, 9.9999040e-01],
       [5.7091602e-05, 9.9986506e-01, 7.7824567e-05]], dtype=float32)

{'accuracy': 85.9504132231405,
 'precision': 0.8602805543527452,
 'recall': 0.859504132231405,
 'f1': 0.8596416741015441}

1/1 [==============================] - 0s 36ms/step


array([0., 1., 0.], dtype=float32)

In [ ]:
predicted_sentiment = []

for idx in range(len(test_tweet)):
  predicted = tf.squeeze(tf.round(lstm3.predict(np.array([test_tweet[idx]])))).numpy()
  predicted_sentiment.append(predicted)

1/1 [==============================] - 0s 37ms/step


In [ ]:
predicted_dataframe['sentimen'] = test_sentimen
predicted_dataframe['predicted_sentimen'] = [np.argmax(array) for array in predicted_sentiment]
predicted_dataframe['tweet'] = test_tweet

display(predicted_dataframe)

,sentimen,predicted_sentimen,tweet
0,1,1,mau tanya impact buat negara kek gimana kalo g...
1,1,1,jokowi raja salman sepakat tingkat kerja sama ...
2,0,0,kala tax amnesty kemarin prabowo sama sandiaga...
3,1,1,gimana nih apa gaji tidak jokowi prabowo
4,1,1,beranganangan gaji dokter binatang minimal jut...
...,...,...,...
721,0,0,nyerocos gurubesar sumitro joyohadikusumo ekon...
722,1,1,asosiasi usaha bumiputera nusantara se indones...
723,0,0,tidak masuk otak lu bapak jokowi nanya ekonomi...
724,0,0,eh abang beda startup monetizing hobi monetize...


In [ ]:
print("Akurasi: ", accuracy_score(predicted_dataframe['sentimen'], predicted_dataframe['predicted_sentimen']))

Akurasi:  0.859504132231405
